In [1]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Download the necessary NLTK data packages
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('punkt_tab') # Download the punkt_tab data package


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amous\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amous\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\amous\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
import json

# Chemin du fichier JSON
document = "chatbot_tourisme_fr_en.json"  # Update the path to the correct location

# Lecture du fichier JSON
with open(document, "r", encoding="utf-8") as f:
    data = json.load(f)

# Affichage du contenu
print(data)


{'intentions': [{'tag': 'salutations', 'questions_fr': ['Bonjour', 'Bonsoir', 'Comment allez-vous ?', 'Ca va .', 'Salut', 'Quoi de neuf', 'Ca va ?'], 'questions_en': ['Hello', 'Good Morning', 'Good afternoon', 'How are you?', "How's it going?", 'Hi', "What's up?", 'How are you doing?'], 'réponses_fr': ['Bonjour et Bienvenue dans le pays de la TERANGA', 'Bonsoir et Bienvenue dans le pays de la TERANGA', 'Oui je vais bien et chez vous?', 'En quoi puis-je vous être utile ?', 'Hey!!Tout se passe bien pour vous ?', 'Oui ca va .Que puis-je faire pour vous aider ?', 'Oui ca va.En quoi puis-je vous être utile ?'], 'réponses_en': ['Hello and welcome to the land of TERANGA.', 'Morning and welcome to the land of TERANGA.', 'Good afternoon and welcome to the land of TERANGA.', "Yes, I'm fine, and you?", 'How can I help you?', 'Hey!! How are things going for you?', "Yes, I'm fine. What can I do to help you?", "Yes, I'm fine. How can I assist you?"]}, {'tag': 'recommandation_destination_senegal', 'q

In [3]:
# initialisation de lemmatizer pour obtenir la racine des mots
lemmatizer = WordNetLemmatizer()

# création des listes
words = []
classes = []
doc_X = []
doc_y = []

# parcourir avec une boucle For toutes les intentions
# tokéniser chaque pattern et ajouter les tokens à la liste words, les patterns et
# le tag associé à l'intention sont ajoutés aux listes correspondantes
# Parcourir toutes les intentions
for intent in data["intentions"]:
    # Traiter les questions françaises ET anglaises
    for lang_key in ["questions_fr", "questions_en"]:  # Ajout des clés linguistiques
        if lang_key in intent:  # Vérifier la présence de la clé
            for question in intent[lang_key]:
                tokens = nltk.word_tokenize(question)
                words.extend(tokens)
                doc_X.append(question)

                # Associer le tag si disponible
                if "tag" in intent:
                    doc_y.append(intent["tag"])

    # Gestion des tags
    if "tag" in intent and intent["tag"] not in classes:
        classes.append(intent["tag"])

# Le reste du code reste inchangé
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [4]:
print(words)

["'s", 'a', 'a-t-il', 'about', 'accessible', 'accessibles', 'acheter', 'activity', 'activités', 'afternoon', 'alimentaires', 'aller', 'allez-vous', 'alone', 'an', 'animal', 'animaux', 'any', 'are', 'area', 'around', 'art', 'artisanat', 'artisanaux', 'artist', 'artiste', 'assister', 'at', 'attend', 'au', 'average', 'avoid', 'back', 'be', 'beach', 'best', 'bird', 'bon', 'bonjour', 'bonsoir', 'bord', 'bring', 'buy', 'by', 'ca', 'can', 'car', 'centre', 'ceremony', 'climat', 'climate', 'clothes', 'colonial', 'comment', 'commerciaux', 'common', 'commun', 'conseiller', 'conseils', 'country', 'courants', 'craft', 'culinaires', 'culinary', 'cultural', 'culturelles', 'culturels', 'célèbre', 'célèbres', 'cérémonies', 'dakar', 'de', 'destination', 'dialect', 'dialectes', 'dire', 'dish', 'do', 'doctor', 'doing', 'du', 'during', 'déplacer', 'détendre', 'easily', 'eat', 'emblématiques', 'en', 'endroits', 'ensure', 'essayer', 'est', 'est-elle', 'est-il', 'event', 'excursion', 'faire', 'famous', 'faune

In [5]:
print(classes)

['activites_senegal', 'artisanat_senegal', 'climat_senegal', 'culture_senegal', 'faune_senegal', 'gastronomie_senegal', 'histoire_senegal', 'langues_senegal', 'musique_senegal', 'plages_senegal', 'recommandation_destination_senegal', 'salutations', 'sante_senegal', 'securite_senegal', 'shopping_senegal', 'sports_senegal', 'transport_senegal']


In [6]:
print(doc_X)

['Bonjour', 'Bonsoir', 'Comment allez-vous ?', 'Ca va .', 'Salut', 'Quoi de neuf', 'Ca va ?', 'Hello', 'Good Morning', 'Good afternoon', 'How are you?', "How's it going?", 'Hi', "What's up?", 'How are you doing?', 'Quelles sont les meilleures destinations au Sénégal ?', 'Où aller au Sénégal pour des vacances ?', 'Pouvez-vous me conseiller une destination au Sénégal ?', 'Quels sont les lieux incontournables au Sénégal ?', 'Quels sont les endroits populaires pour les touristes au Sénégal ?', 'What are the best destinations in Senegal?', 'Where to go in Senegal for a vacation?', 'Can you recommend a destination in Senegal?', 'What are the must-see places in Senegal?', 'What are the popular tourist spots in Senegal?', 'Quelles activités peut-on faire au Sénégal ?', 'Que faire pendant mon séjour au Sénégal ?', 'Pouvez-vous me conseiller des excursions au Sénégal ?', 'Quels musées visiter au Sénégal ?', 'Quels sont les festivals culturels au Sénégal ?', 'What activities can you do in Senegal

In [7]:
print(doc_y)

['salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'salutations', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'recommandation_destination_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'activites_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 'gastronomie_senegal', 

In [8]:
# liste pour les données d'entraînement
training = []
out_empty = [0] * len(classes)

# création du modèle d'ensemble de mots
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    # marque l'index de la classe à laquelle le pattern atguel est associé à
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    # ajoute le one hot encoded BoW et les classes associées à la liste training
    training.append([bow, output_row])

# mélanger les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)

# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [9]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200

In [10]:
# modèle Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               44800     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 17)                1105      
                                                                 
Total params: 54,161
Trainable params: 54,161
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# entraînement du modèle
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
6/6 [==============================] - 0s 2ms/step - loss: 2.8906 - accuracy: 0.0904
Epoch 2/200
6/6 [==============================] - 0s 1ms/step - loss: 2.7163 - accuracy: 0.1525
Epoch 3/200
6/6 [==============================] - 0s 1ms/step - loss: 2.5090 - accuracy: 0.2825
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 2.1890 - accuracy: 0.3842
Epoch 5/200
6/6 [==============================] - 0s 1000us/step - loss: 1.7562 - accuracy: 0.5085
Epoch 6/200
6/6 [==============================] - 0s 1000us/step - loss: 1.5613 - accuracy: 0.5141
Epoch 7/200
6/6 [==============================] - 0s 999us/step - loss: 1.0434 - accuracy: 0.7062
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 0.9020 - accuracy: 0.7401
Epoch 9/200
6/6 [==============================] - 0s 1000us/step - loss: 0.8316 - accuracy: 0.7740
Epoch 10/200
6/6 [==============================] - 0s 1ms/step - loss: 0.5018 - accuracy: 0.8475
Epoch 11/200
6/6 [

In [13]:
!pip install langdetect

In [14]:
from langdetect import detect
import random
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list



import difflib

def find_best_match(user_question, questions_list):
    best_match = difflib.get_close_matches(user_question, questions_list, n=1, cutoff=0.6)
    return best_match[0] if best_match else None

def get_response(intents_list, intents_json, user_question):
    try:
        user_language = detect(user_question)
    except:
        user_language = "fr"

    language = "fr" if user_language.startswith("fr") else "en"

    tag = intents_list[0] if intents_list else None
    if not tag:
        return "Je ne comprends pas."

    for intent in intents_json["intentions"]:
        if intent["tag"] == tag:
            questions_key = f"questions_{language}"
            responses_key = f"réponses_{language}"

            if questions_key in intent and responses_key in intent:
                best_match = find_best_match(user_question, intent[questions_key])
                if best_match:
                    question_index = intent[questions_key].index(best_match)
                    return intent[responses_key][question_index]
                else:
                    return random.choice(intent[responses_key])
    return "Je ne comprends pas."





In [15]:
import pickle
# Sauvegarder le modèle
model.save('model.h5')

# Sauvegarder les données de prétraitement
preprocessing_data = {
    'words': words,
    'classes': classes
}
with open('preprocessing_data.pkl', 'wb') as f:
    pickle.dump(preprocessing_data, f)
    

In [ ]:
while True:
    message = input("")
    if message.lower() in ["exit", "quit", "stop"]:
        print("Chatbot arrêté. À bientôt !")
        break
    intents = pred_class(message, words, classes)
    # Pass the user's message (message) as the user_question argument
    result = get_response(intents, data, message)
    print(result)